In [1]:
import pickle
import h5py
import os
from sklearn import svm
from statistics import mean
from random import shuffle
import numpy as np
from sklearn.utils import shuffle as shf
from sklearn.neural_network import MLPClassifier
import graphviz

In [2]:
data2 = {}
print('loading data')
for key in os.listdir('E:/Time Series/hdf_files'):
	filename = 'E:/Time Series/hdf_files/'+ key
	f = h5py.File(filename, 'r')
	data2[key] = list(list(f.values())[0])
	f.close()

loading data


In [5]:
data20 = data2

In [6]:
for key, value in data20.items():
    print(key, '---', len(value))
    shuffle(data20[key])

ALFALFA --- 254001
CORN --- 6224142
DEVELOPED --- 878386
FOREST --- 766945
OATS --- 12655
RYE --- 5122
SOYABEENS --- 5960145
WATER_BODY --- 37743


In [7]:
data = {key:value[:10000] for key, value in data20.items() if len(value) >= 20000}

In [17]:
for key in data.keys():
    for n in range(len(data[key])):
        example = list(data[key][n])
        for i in range(45):
            example.append(example[i]*example[i])
        data[key][n] = example

C:\Users\prabh\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: RuntimeWarning: overflow encountered in ushort_scalars
  """


In [8]:
mins = {}; maxs = {}; means = {}
for i in range(45):
    lin = [l[i] for value in data.values() for l in value]
    mins[i] = min(lin)
    maxs[i] = max(lin)
    means[i] = mean(lin)

In [9]:
data = {key: [[np.float64((example[i]-means[i])/(maxs[i] - mins[i])) for i in range(45)] for example in value] for key, value in data.items()}

C:\Users\prabh\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: RuntimeWarning: overflow encountered in ushort_scalars
  """Entry point for launching an IPython kernel.


In [10]:
data3 = data #basic data

In [19]:
data4 = data #with quadratic terms

In [8]:
data = data3

In [11]:
for key in data.keys():
    shuffle(data[key])

In [12]:
training_examples = []
training_labels = []
test_examples = []
test_labels = []
cv_examples = []
cv_labels = []
i = 0;
for key, value in data.items():
    training_examples = training_examples + value
    #cv_examples = cv_examples + value[18000:19000]
    #test_examples = test_examples + value[19000:]
    training_labels = training_labels + [i for v in range(len(value))]
    #cv_labels = cv_labels + [i for v in range(1000)]
    #test_labels = test_labels + [i for v in range(1000)]
    print(i,':', key)
    i+=1

0 : ALFALFA
1 : CORN
2 : DEVELOPED
3 : FOREST
4 : SOYABEENS
5 : WATER_BODY


In [13]:
training_examples, training_labels  = shf(training_examples+cv_examples, training_labels+cv_labels)

In [14]:
from sklearn import metrics
from sklearn.ensemble import ExtraTreesClassifier
model = ExtraTreesClassifier()
model.fit(training_examples, training_labels)

ExtraTreesClassifier(bootstrap=False, class_weight=None, criterion='gini',
           max_depth=None, max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [25]:
s = model.feature_importances_
L = np.array(s)
t = sorted(range(len(L)), key=lambda i:L[i])
z = []; r = 0
for j in t[::-1]:
    print( j, '-----', s[j])
    z.append(j)
    s

44 ----- 0.05689138342908255
34 ----- 0.044961685771263665
39 ----- 0.04445599606430319
4 ----- 0.04336965191784822
40 ----- 0.040217375392934206
15 ----- 0.03343321257797839
14 ----- 0.03312164965007875
24 ----- 0.03234844713744776
10 ----- 0.031109301369145505
30 ----- 0.030324439276916394
29 ----- 0.024572590135006546
0 ----- 0.024159223188578306
19 ----- 0.023620707522889077
37 ----- 0.02225816152045502
2 ----- 0.02225671151652781
5 ----- 0.021294964626234984
25 ----- 0.021027147984044554
1 ----- 0.021008694150807847
12 ----- 0.020488233003941904
9 ----- 0.02025907833310559
23 ----- 0.018795116450663847
7 ----- 0.018513992354528816
17 ----- 0.018398574208920437
22 ----- 0.018214200295697935
26 ----- 0.01816420536339443
20 ----- 0.017931315438434577
35 ----- 0.017619148486262986
27 ----- 0.01685447587184389
36 ----- 0.01668565262310511
11 ----- 0.016467602229886025
42 ----- 0.016453445272418633
32 ----- 0.01591047528966951
13 ----- 0.015237730426668733
43 ----- 0.015008101983340072


In [86]:
data = {key:[[example[i] for i in z]for example in value] for key, value in data.items()}
print(len(z))

22


In [10]:
training_examples = []
training_labels = []
test_examples = []
test_labels = []
cv_examples = []
cv_labels = []
i = 0;
for key, value in data.items():
    training_examples = training_examples + value[:18000]
    cv_examples = cv_examples + value[18000:19000]
    test_examples = test_examples + value[19000:]
    training_labels = training_labels + [i for v in range(18000)]
    cv_labels = cv_labels + [i for v in range(1000)]
    test_labels = test_labels + [i for v in range(1000)]
    print(i,':', key)
    i+=1

0 : ALFALFA
1 : CORN
2 : DEVELOPED
3 : FOREST
4 : SOYABEENS
5 : WATER_BODY


In [15]:
training_examples, training_labels  = shf(training_examples, training_labels)

In [28]:
clf = svm.SVC(C=100000,
              kernel='rbf', 
              degree= 3,
              gamma= 'auto',
              coef0=0.0,
              shrinking=True, 
              probability= False, 
              tol=0.0000001, 
              cache_size=200, 
              class_weight=None, 
              verbose=True, 
              max_iter=-1, 
              decision_function_shape='ovr', 
              random_state=None)


In [ ]:
clf.fit(training_examples, training_labels)

[LibSVM]

In [152]:
sum(clf.predict(cv_examples) == cv_labels)/len(cv_labels)

0.714

In [153]:
sum(clf.predict(training_examples) == training_labels)/len(training_labels)

0.87416

In [144]:
for i in clf.predict(training_examples):
    if i == 5:print (i)

In [145]:
import sklearn
sklearn.metrics.log_loss(training_labels, clf.predict(training_examples), eps=1e-15, normalize=True, sample_weight=None, labels=None)

ValueError: y_true and y_pred contain different number of classes 5, 2. Please provide the true labels explicitly through the labels argument. Classes found in y_true: [0 1 2 3 4]

In [ ]:
sklearn.metrics.log_loss(cv_labels, clf.predict(cv_labels), eps=1e-15, normalize=True, sample_weight=None, labels=None)

In [ ]:
sum(clf.predict(test_examples) == test_labels)/len(test_labels)

In [61]:
file_object = open('clf.pkl', 'wb')
pickle.dump(clf, file_object)
file_object.close()

In [16]:
cnf = MLPClassifier(hidden_layer_sizes=(1000,1000,1000), activation='tanh', solver='sgd',
                    alpha=0.00000001,   batch_size= 10000,    learning_rate='adaptive',
                    learning_rate_init=0.0001,    power_t=0.5,    max_iter=10000, 
                    shuffle=True,     random_state=None,  tol=0.00000001, 
                    verbose=True,     warm_start= True,   momentum=0.9, 
                    nesterovs_momentum=True, early_stopping=False, validation_fraction=0.1, 
                    beta_1=0.5, 
                    beta_2=0.5, 
                    epsilon=1e-10
                   ) 

In [ ]:
cnf.fit(training_examples, training_labels)

Iteration 1, loss = 1.80630957
Iteration 2, loss = 1.78342721
Iteration 3, loss = 1.75333070
Iteration 4, loss = 1.72221404
Iteration 5, loss = 1.69299767
Iteration 6, loss = 1.66644719
Iteration 7, loss = 1.64295931
Iteration 8, loss = 1.62256003
Iteration 9, loss = 1.60487903
Iteration 10, loss = 1.58944278
Iteration 11, loss = 1.57598698
Iteration 12, loss = 1.56424912
Iteration 13, loss = 1.55382972
Iteration 14, loss = 1.54457335
Iteration 15, loss = 1.53626481
Iteration 16, loss = 1.52878263
Iteration 17, loss = 1.52192896
Iteration 18, loss = 1.51569289
Iteration 19, loss = 1.50991322
Iteration 20, loss = 1.50460364
Iteration 21, loss = 1.49964337
Iteration 22, loss = 1.49503515
Iteration 23, loss = 1.49070829
Iteration 24, loss = 1.48662398
Iteration 25, loss = 1.48277482
Iteration 26, loss = 1.47917529
Iteration 27, loss = 1.47572785
Iteration 28, loss = 1.47244370
Iteration 29, loss = 1.46934084
Iteration 30, loss = 1.46637131
Iteration 31, loss = 1.46353681
Iteration 32, los

Iteration 253, loss = 1.33201143
Iteration 254, loss = 1.33185127
Iteration 255, loss = 1.33168824
Iteration 256, loss = 1.33153277
Iteration 257, loss = 1.33138084
Iteration 258, loss = 1.33121682
Iteration 259, loss = 1.33105980
Iteration 260, loss = 1.33090835
Iteration 261, loss = 1.33075171
Iteration 262, loss = 1.33060070
Iteration 263, loss = 1.33044549
Iteration 264, loss = 1.33029271
Iteration 265, loss = 1.33014464
Iteration 266, loss = 1.32999337
Iteration 267, loss = 1.32984529
Iteration 268, loss = 1.32969590
Iteration 269, loss = 1.32954869
Iteration 270, loss = 1.32940289
Iteration 271, loss = 1.32925575
Iteration 272, loss = 1.32911137
Iteration 273, loss = 1.32896632
Iteration 274, loss = 1.32882286
Iteration 275, loss = 1.32867951
Iteration 276, loss = 1.32853712
Iteration 277, loss = 1.32839703
Iteration 278, loss = 1.32825547
Iteration 279, loss = 1.32811463
Iteration 280, loss = 1.32797663
Iteration 281, loss = 1.32783857
Iteration 282, loss = 1.32770885
Iteration 

Iteration 502, loss = 1.30827675
Iteration 503, loss = 1.30821544
Iteration 504, loss = 1.30815807
Iteration 505, loss = 1.30809688
Iteration 506, loss = 1.30804212
Iteration 507, loss = 1.30798992
Iteration 508, loss = 1.30792695
Iteration 509, loss = 1.30787298
Iteration 510, loss = 1.30781549
Iteration 511, loss = 1.30775781
Iteration 512, loss = 1.30769969
Iteration 513, loss = 1.30764239
Iteration 514, loss = 1.30758635
Iteration 515, loss = 1.30753600
Iteration 516, loss = 1.30748117
Iteration 517, loss = 1.30741990
Iteration 519, loss = 1.30730885
Iteration 520, loss = 1.30726035
Iteration 521, loss = 1.30720017
Iteration 522, loss = 1.30714670
Iteration 523, loss = 1.30709293
Iteration 524, loss = 1.30703735
Iteration 525, loss = 1.30698370
Iteration 526, loss = 1.30692700
Iteration 527, loss = 1.30688116
Iteration 528, loss = 1.30681704
Iteration 529, loss = 1.30677010
Iteration 530, loss = 1.30671632
Iteration 531, loss = 1.30665756
Iteration 532, loss = 1.30660250
Iteration 

Iteration 752, loss = 1.29753815
Iteration 753, loss = 1.29750637
Iteration 754, loss = 1.29747437
Iteration 755, loss = 1.29744027
Iteration 756, loss = 1.29740502
Iteration 757, loss = 1.29737700
Iteration 758, loss = 1.29734270
Iteration 759, loss = 1.29731328
Iteration 760, loss = 1.29727841
Iteration 761, loss = 1.29724747
Iteration 762, loss = 1.29721624
Iteration 763, loss = 1.29718533
Iteration 764, loss = 1.29715446
Iteration 765, loss = 1.29712262
Iteration 766, loss = 1.29708983
Iteration 767, loss = 1.29706274
Iteration 768, loss = 1.29702963
Iteration 769, loss = 1.29699643
Iteration 770, loss = 1.29697045
Iteration 771, loss = 1.29693589
Iteration 772, loss = 1.29690188
Iteration 773, loss = 1.29686974
Iteration 774, loss = 1.29684173
Iteration 775, loss = 1.29681190
Iteration 776, loss = 1.29677821
Iteration 777, loss = 1.29674986
Iteration 778, loss = 1.29672084
Iteration 779, loss = 1.29669007
Iteration 780, loss = 1.29665701
Iteration 781, loss = 1.29662861
Iteration 

Iteration 1001, loss = 1.29096791
Iteration 1002, loss = 1.29094670
Iteration 1003, loss = 1.29092368
Iteration 1004, loss = 1.29090639
Iteration 1005, loss = 1.29088350
Iteration 1006, loss = 1.29085953
Iteration 1007, loss = 1.29083574
Iteration 1008, loss = 1.29082015
Iteration 1009, loss = 1.29079438
Iteration 1010, loss = 1.29077261
Iteration 1011, loss = 1.29075213
Iteration 1012, loss = 1.29073453
Iteration 1013, loss = 1.29071491
Iteration 1014, loss = 1.29068467
Iteration 1015, loss = 1.29066277
Iteration 1016, loss = 1.29064654
Iteration 1017, loss = 1.29062341
Iteration 1018, loss = 1.29060076
Iteration 1019, loss = 1.29058204
Iteration 1020, loss = 1.29056180
Iteration 1021, loss = 1.29053829
Iteration 1022, loss = 1.29051751
Iteration 1023, loss = 1.29049986
Iteration 1024, loss = 1.29047499
Iteration 1025, loss = 1.29045156
Iteration 1026, loss = 1.29043253
Iteration 1027, loss = 1.29041734
Iteration 1028, loss = 1.29038760
Iteration 1029, loss = 1.29036905
Iteration 1030

...

Iteration 2206, loss = 1.27456107
Iteration 2207, loss = 1.27456820
Iteration 2208, loss = 1.27454996
Iteration 2209, loss = 1.27453341
Iteration 2210, loss = 1.27452565
Iteration 2211, loss = 1.27451655
Iteration 2212, loss = 1.27451063
Iteration 2213, loss = 1.27450157
Iteration 2214, loss = 1.27449434
Iteration 2215, loss = 1.27448232
Iteration 2216, loss = 1.27446726
Iteration 2217, loss = 1.27446102
Iteration 2218, loss = 1.27445172
Iteration 2219, loss = 1.27444268
Iteration 2220, loss = 1.27443871
Iteration 2221, loss = 1.27442260
Iteration 2222, loss = 1.27441884
Iteration 2223, loss = 1.27439826
Iteration 2224, loss = 1.27439468
Iteration 2225, loss = 1.27438340
Iteration 2226, loss = 1.27437201
Iteration 2227, loss = 1.27436821
Iteration 2228, loss = 1.27435194
Iteration 2229, loss = 1.27435135
Iteration 2230, loss = 1.27433596
Iteration 2231, loss = 1.27432414
Iteration 2232, loss = 1.27431575
Iteration 2233, loss = 1.27430864
Iteration 2234, loss = 1.27429759
Iteration 2235

Iteration 2447, loss = 1.27233301
Iteration 2448, loss = 1.27232674
Iteration 2449, loss = 1.27231530
Iteration 2450, loss = 1.27230846
Iteration 2451, loss = 1.27229976
Iteration 2452, loss = 1.27228750
Iteration 2453, loss = 1.27228655
Iteration 2454, loss = 1.27227098
Iteration 2455, loss = 1.27226630
Iteration 2456, loss = 1.27226602
Iteration 2457, loss = 1.27224299
Iteration 2458, loss = 1.27223481
Iteration 2459, loss = 1.27222935
Iteration 2460, loss = 1.27221639
Iteration 2461, loss = 1.27220556
Iteration 2462, loss = 1.27219963
Iteration 2463, loss = 1.27219456
Iteration 2464, loss = 1.27218129
Iteration 2465, loss = 1.27217376
Iteration 2466, loss = 1.27216518
Iteration 2467, loss = 1.27215589
Iteration 2468, loss = 1.27214338
Iteration 2469, loss = 1.27213766
Iteration 2470, loss = 1.27213567
Iteration 2471, loss = 1.27211985
Iteration 2472, loss = 1.27211623
Iteration 2473, loss = 1.27210008
Iteration 2474, loss = 1.27209052
Iteration 2475, loss = 1.27208362
Iteration 2476

Iteration 2688, loss = 1.27024607
Iteration 2689, loss = 1.27023505
Iteration 2690, loss = 1.27022870
Iteration 2691, loss = 1.27022156
Iteration 2692, loss = 1.27021123
Iteration 2693, loss = 1.27020400
Iteration 2694, loss = 1.27019739
Iteration 2695, loss = 1.27019372
Iteration 2696, loss = 1.27017783
Iteration 2697, loss = 1.27017009
Iteration 2698, loss = 1.27016288
Iteration 2699, loss = 1.27015170
Iteration 2700, loss = 1.27014362
Iteration 2701, loss = 1.27013608
Iteration 2702, loss = 1.27012638
Iteration 2703, loss = 1.27012312
Iteration 2704, loss = 1.27011517
Iteration 2705, loss = 1.27010853
Iteration 2706, loss = 1.27009647
Iteration 2707, loss = 1.27008690
Iteration 2708, loss = 1.27007449
Iteration 2709, loss = 1.27007417
Iteration 2710, loss = 1.27006293
Iteration 2711, loss = 1.27005272
Iteration 2712, loss = 1.27004517
Iteration 2713, loss = 1.27003268
Iteration 2714, loss = 1.27002426
Iteration 2715, loss = 1.27001774
Iteration 2716, loss = 1.27001119
Iteration 2717

Iteration 2929, loss = 1.26826272
Iteration 2930, loss = 1.26825171
Iteration 2931, loss = 1.26824674
Iteration 2932, loss = 1.26824748
Iteration 2933, loss = 1.26823148
Iteration 2934, loss = 1.26822582
Iteration 2935, loss = 1.26821693
Iteration 2936, loss = 1.26820880
Iteration 2937, loss = 1.26820183
Iteration 2938, loss = 1.26819306
Iteration 2939, loss = 1.26819391
Iteration 2940, loss = 1.26817156
Iteration 2941, loss = 1.26816953
Iteration 2942, loss = 1.26815569
Iteration 2943, loss = 1.26815341
Iteration 2944, loss = 1.26814056
Iteration 2945, loss = 1.26813600
Iteration 2946, loss = 1.26813197
Iteration 2947, loss = 1.26811870
Iteration 2948, loss = 1.26811042
Iteration 2949, loss = 1.26810743
Iteration 2950, loss = 1.26809915
Iteration 2951, loss = 1.26808819
Iteration 2952, loss = 1.26807596
Iteration 2953, loss = 1.26807386
Iteration 2954, loss = 1.26806625
Iteration 2955, loss = 1.26805532
Iteration 2956, loss = 1.26804639
Iteration 2957, loss = 1.26804113
Iteration 2958

Iteration 3170, loss = 1.26636709
Iteration 3171, loss = 1.26635715
Iteration 3172, loss = 1.26634786
Iteration 3173, loss = 1.26633900
Iteration 3174, loss = 1.26634394
Iteration 3175, loss = 1.26632148
Iteration 3176, loss = 1.26631332
Iteration 3177, loss = 1.26630801
Iteration 3178, loss = 1.26630060
Iteration 3179, loss = 1.26629365
Iteration 3180, loss = 1.26628763
Iteration 3181, loss = 1.26628246
Iteration 3182, loss = 1.26627885
Iteration 3183, loss = 1.26626917
Iteration 3184, loss = 1.26625748
Iteration 3185, loss = 1.26624771
Iteration 3186, loss = 1.26624145
Iteration 3187, loss = 1.26623103
Iteration 3188, loss = 1.26623201
Iteration 3189, loss = 1.26621794
Iteration 3190, loss = 1.26621095
Iteration 3191, loss = 1.26620214
Iteration 3192, loss = 1.26619695
Iteration 3193, loss = 1.26618863
Iteration 3194, loss = 1.26617682
Iteration 3195, loss = 1.26616753
Iteration 3196, loss = 1.26615913
Iteration 3197, loss = 1.26615816
Iteration 3198, loss = 1.26615914
Iteration 3199

In [ ]:
sum(cnf.predict(training_examples) == training_labels)/len(training_labels)

In [ ]:
cnf = MLPClassifier(hidden_layer_sizes=(1000,1000,1000), activation='tanh', solver='sgd',
                    alpha=0.00000001,   batch_size= 10000,    learning_rate='adaptive',
                    learning_rate_init=0.0001,    power_t=0.5,    max_iter=10000, 
                    shuffle=True,     random_state=None,  tol=0.00000001, 
                    verbose=True,     warm_start= True,   momentum=0.9, 
                    nesterovs_momentum=True, early_stopping=False, validation_fraction=0.1, 
                    beta_1=0.5, 
                    beta_2=0.5, 
                    epsilon=1e-10
                   )
cnf.fit(training_examples, training_labels)
sum(cnf.predict(training_examples) == training_labels)/len(training_labels)